# Introduction

Some example code showing how to automatically download files from a Google drive folder, then load all csv and excel into a Postgres database. The code will handle csv encoding issues as well as load all tabs in each Excel file. 

However, it is only an example and would need more work because:

- Some Excel files have more than one table on a sheet
- Many Excel files have blank rows, you'd need to scan through them with openpyxl and remove so column heading are identified
- Table names are cleaned <FILE NAME>_<SHEET NAME IF EXCEL WITH MULTIPLE SHEETS>

The examples chosen are for the Save the Children stream of the DataKind 2022 DataDive.

- [project brief](https://docs.google.com/document/d/1TQ2TiGK_k8KEIUPzVb3ZSKaxzEBP9s9ZJBI2U_HeQ6U/edit#)
- [Google drive with data](https://drive.google.com/drive/folders/1G_CAhpb0xV9zRrV-T5ngA03x-IZ4tGKz)

The example is meant to show how we might load all data into a database to see how we might link data together, build classification and persist meta data. There are many other ways of course, but good 'ol SQL might be a good start!

## Setup

1. Go to the DataDive [google folder](https://drive.google.com/drive/folders/1eBQZhSeCXC9a62Dpg8_cWeKIZXfrxfGz)
2. Right-click on a sub-folder and Create a shortcut in your My Drive, eg for Kenya folder
3. Run pane to mount google drive below, connect your drive
5. Edit `dir` variable below and set path as seen in the laft pane

In [ ]:
# If you don't have these installed, uncomment these lines and run
#%pip install gdown
#%pip install chardet
#%pip install sqlalchemy
#%pip install psycopg2-binary

In [ ]:
import gdown
import os
import pandas as pd
import chardet

## Examples
### Downloading a folder from gdrive

In [17]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
#folder_url = "https://drive.google.com/drive/folders/1G_CAhpb0xV9zRrV-T5ngA03x-IZ4tGKz?usp=share_link"
#gdown.download_folder(folder_url, quiet=True, use_cookies=False)

In [156]:
def load_data(dir, dfs, country):
  for f in os.listdir(dir):
    file = f
    f = f"{dir}/{f}"
    if '.csv' in f:
        print(f"Loading csv file {f}")
        # Detect encoding
        with open(f, 'rb') as rawdata:
            r = chardet.detect(rawdata.read(100000))
        df = pd.read_csv(f, encoding=r['encoding'])
        df['file'] = file
        df['sheet'] = ''
        df['country'] = country
        dfs[f] = df
    if '.xlsx' in f:
        print(f"Leading {f} ...")
        # Read all sheets in
        sheet_to_df_map = pd.read_excel(f, sheet_name=None)
        for sheet in sheet_to_df_map:
          # TODO
          # 1. TEST FOR BLANK ROWS AT TOP, AND REMOVE
          # 2. Does the table have a blank column, if so, is it two tables on one sheet. If so handle that
          df = sheet_to_df_map[sheet]
          df['file'] = file
          df['sheet'] = sheet
          df['country'] = country
          dfs[f"{f} - SHEET= {sheet}"] = df 
  print("Done")
  return dfs

def output_data_summaries(dfs, meta_data):
  op_meta_data = []
  for key in dfs:
      print("\n\n===================================================================================================\n\n")
      df = dfs[key]
      print(key)
      f = str(df['file'])
      country = str(df['country'])
      print(f"Country: {country} > File + Sheet: {key} > Size: {df.shape}\n\n")
      file_meta = meta_data.loc[meta_data[file_col] == f]
      description = str(file_meta['Dataset description'])
      link = str(file_meta['Dataset'])
      print("Link: ", str(link),"\n\n")
      print("Description: \n\n",str(description),"\n\n")
      display(df)
  return 

def create_meta_data(dfs, meta_data):
  op_meta_data = []
  for key in dfs:
      df = dfs[key]
      if isinstance(df, pd.DataFrame) and df.shape[0] > 0:
        f = str(df['file'])
        country = str(df['country'][0])
        file_meta = meta_data.loc[meta_data[file_col] == f]
        description = str(file_meta['Dataset description'])
        link = str(file_meta['Dataset'])
        r = {
          "key":key,
          "file":f,
          "description":str(description),
          "country":str(df['country'][0]),
          "rows": str(df.shape[0]),
          "columns": str(df.shape[1]),
          "data_columns": str(list(df.columns)),
          "data_types": str(list(df.dtypes))
        }
      else:
        r = {
          "key": key,
          "file": "",
          "description":"",
          "country":"",
          "rows": "",
          "columns": "",
          "data_columns": "",
          "data_types":""
        }
      op_meta_data.append(r)
  return pd.DataFrame(op_meta_data)

In [159]:
op_meta_data = create_meta_data(dfs, meta_data)
display(op_meta_data)
op_meta_data.to_csv('/content/drive/MyDrive/Kenya/ALL_META_DATA.csv')

,key,file,description,country,rows,columns,data_columns,data_types
0,/content/drive/MyDrive/Kenya/kenya-3w-final-li...,0 kenya-3w-final-list-2017.xlsx\n1 ...,"Series([], Name: Dataset description, dtype: o...",Kenya,1008,8,"['COUNTY', 'NAME', 'TYPE', 'ABBREV', 'SECTOR',...","[dtype('O'), dtype('O'), dtype('O'), dtype('O'..."
1,/content/drive/MyDrive/Kenya/kenya-3w-final-li...,0 kenya-3w-final-list-2017.xlsx\n1 ken...,"Series([], Name: Dataset description, dtype: o...",Kenya,42,17,"['COUNTY', 'COUNTY.1', 'DRR', 'Education', 'Em...","[dtype('O'), dtype('O'), dtype('float64'), dty..."
2,/content/drive/MyDrive/Kenya/kenya-3w-final-li...,0 kenya-3w-final-list-2017.xlsx\n1 ken...,"Series([], Name: Dataset description, dtype: o...",Kenya,42,6,"['COUNTY', 'COUNTY.1', 'Food', 'file', 'sheet'...","[dtype('O'), dtype('O'), dtype('float64'), dty..."
3,/content/drive/MyDrive/Kenya/kenya-3w-final-li...,0 kenya-3w-final-list-2017.xlsx\n1 ken...,"Series([], Name: Dataset description, dtype: o...",Kenya,65,10,"['Organization', 'Abbrev', 'Type', 'Unnamed: 3...","[dtype('O'), dtype('O'), dtype('O'), dtype('fl..."
4,/content/drive/MyDrive/Kenya/kenya-3w-final-li...,0 kenya-3w-final-list-2017.xlsx\n1 ken...,"Series([], Name: Dataset description, dtype: o...",Kenya,35,6,"['Name', 'Abbred', 'Type', 'file', 'sheet', 'c...","[dtype('O'), dtype('O'), dtype('O'), dtype('O'..."
...,...,...,...,...,...,...,...,...
96,/content/drive/MyDrive/Somalia/Somalia-Humanit...,0 Somalia-Humanitarian Needs Overview-2020...,"Series([], Name: Dataset description, dtype: o...",Somalia,24,26,"['Well being', 'Unnamed: 1', 'Unnamed: 2', 'Un...","[dtype('O'), dtype('O'), dtype('O'), dtype('O'..."
97,/content/drive/MyDrive/Somalia/Somalia-Humanit...,0 Somalia-Humanitarian Needs Overview-2020...,"Series([], Name: Dataset description, dtype: o...",Somalia,76,51,"['District', 'Baseline Non-displaced Populatio...","[dtype('O'), dtype('O'), dtype('O'), dtype('O'..."
98,/content/drive/MyDrive/Somalia/Somalia-Humanit...,0 Somalia-Humanitarian Needs Overview-2020...,"Series([], Name: Dataset description, dtype: o...",Somalia,78,31,['HUMANITARIAN CONSEQUENCE: Physical and menta...,"[dtype('O'), dtype('O'), dtype('O'), dtype('O'..."
99,/content/drive/MyDrive/Somalia/Somalia-Humanit...,0 Somalia-Humanitarian Needs Overview-2020...,"Series([], Name: Dataset description, dtype: o...",Somalia,75,31,['HUMANITARIAN CONSEQUENCE: Physical and menta...,"[dtype('O'), dtype('O'), dtype('O'), dtype('O'..."


## Reading files in a directory to pandas

This snippet will read all csv and xls files in a directory into pandas dataframes, create a list of them.

Just an example, needs a bit more logic to handle xlsx sheets which hold multiple tables, blank rows etc.

In [124]:
meta_data = pd.read_csv(f"/content/drive/MyDrive/Kenya/Data metadata - Sheet1.csv")
file_col = 'Google Drive or Github link to downloaded data (if you can put what you’ve downloaded into a shared location!)'

# Disctionary of pandas dataframes
dfs = {}

for country in ["Kenya", "Somalia"]:
  dir = f"/content/drive/MyDrive/{country}"
  dfs = load_data(dir, dfs, country)

  dir = f"/content/drive/MyDrive/{country}"
  dfs = load_data(dir, dfs, country)

print("Done")

Leading /content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx ...


/usr/local/lib/python3.8/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.8/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Leading /content/drive/MyDrive/Kenya/Kenya - IPC Analysis 2017-2022.xlsx ...
Loading csv file /content/drive/MyDrive/Kenya/wfp_food_prices_ken.csv
Leading /content/drive/MyDrive/Kenya/retail-prices-for-dry-beans-2017-per-kg.xlsx ...
Leading /content/drive/MyDrive/Kenya/retail-prices-for-dry-beans-2018-per-kg.xlsx ...
Leading /content/drive/MyDrive/Kenya/retail-prices-for-dry-beans-2019-per-kg.xlsx ...
Leading /content/drive/MyDrive/Kenya/value-of-recorded-marketed-agricultural-production-at-current-prices-2014-2018.xlsx ...
Leading /content/drive/MyDrive/Kenya/ken_admpop_2019.xlsx ...
Loading csv file /content/drive/MyDrive/Kenya/dbo-foodconsumptionscores.csv
Leading /content/drive/MyDrive/Kenya/per-capita-food-consumption-2017-and-2018.xlsx ...
Leading /content/drive/MyDrive/Kenya/ken_adminboundaries_tabulardata.xlsx ...
Loading csv file /content/drive/MyDrive/Kenya/ken_admpop_adm0_2019.csv
Loading csv file /content/drive/MyDrive/Kenya/ken_admpop_adm1_2019.csv
Loading csv file /conten

In [138]:
op_meta_data = create_meta_data(dfs, meta_data)
display(op_meta_data)


/content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx - SHEET= LIST
/content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx - SHEET= FOR IMPORT
/content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx - SHEET=  food import
/content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx - SHEET= Org Lookup
/content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx - SHEET= ABBREV
/content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx - SHEET= Pivot
/content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx - SHEET= County - Abbrev
/content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx - SHEET= Sheet2
/content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx - SHEET= wash
/content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx - SHEET= shelter
/content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx - SHEET= protection
/content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx - SHEET= nutrition
/content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx - SHEET= health

,0
0,0 kenya-3w-final-list-2017.xlsx\n1 ...
1,0 kenya-3w-final-list-2017.xlsx\n1 ken...
2,0 kenya-3w-final-list-2017.xlsx\n1 ken...
3,0 kenya-3w-final-list-2017.xlsx\n1 ken...
4,0 kenya-3w-final-list-2017.xlsx\n1 ken...
...,...
96,0 Somalia-Humanitarian Needs Overview-2020...
97,0 Somalia-Humanitarian Needs Overview-2020...
98,0 Somalia-Humanitarian Needs Overview-2020...
99,0 Somalia-Humanitarian Needs Overview-2020...


In [109]:
output_data_summaries(dfs, meta_data)
display(op_meta_data)





/content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx - SHEET= LIST
Country: 0       Kenya
1       Kenya
2       Kenya
3       Kenya
4       Kenya
        ...  
1003    Kenya
1004    Kenya
1005    Kenya
1006    Kenya
1007    Kenya
Name: country, Length: 1008, dtype: object > File + Sheet: /content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx - SHEET= LIST > Size: (1008, 8)


Link:  Series([], Name: Dataset, dtype: object) 


Description: 

 Series([], Name: Dataset description, dtype: object) 




,COUNTY,NAME,TYPE,ABBREV,SECTOR,file,sheet,country
0,Bungoma,United Nations Childrens Fund,UN,UNICEF,Health,kenya-3w-final-list-2017.xlsx,LIST,Kenya
1,Bungoma,United Nations Childrens Fund,UN,UNICEF,Education,kenya-3w-final-list-2017.xlsx,LIST,Kenya
2,Bungoma,United Nations Childrens Fund,UN,UNICEF,Nutrition,kenya-3w-final-list-2017.xlsx,LIST,Kenya
3,Bungoma,United Nations Childrens Fund,UN,UNICEF,Protection,kenya-3w-final-list-2017.xlsx,LIST,Kenya
4,Bungoma,United Nations Childrens Fund,UN,UNICEF,Emergency Response,kenya-3w-final-list-2017.xlsx,LIST,Kenya
...,...,...,...,...,...,...,...,...
1003,Tharaka - Nithi,World Food Programme,UN,WFP,Education,kenya-3w-final-list-2017.xlsx,LIST,Kenya
1004,Embu,World Food Programme,UN,WFP,Education,kenya-3w-final-list-2017.xlsx,LIST,Kenya
1005,Isiolo,World Food Programme,UN,WFP,Emergency Response,kenya-3w-final-list-2017.xlsx,LIST,Kenya
1006,Samburu,World Food Programme,UN,WFP,Nutrition,kenya-3w-final-list-2017.xlsx,LIST,Kenya






/content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx - SHEET= FOR IMPORT
Country: 0     Kenya
1     Kenya
2     Kenya
3     Kenya
4     Kenya
5     Kenya
6     Kenya
7     Kenya
8     Kenya
9     Kenya
10    Kenya
11    Kenya
12    Kenya
13    Kenya
14    Kenya
15    Kenya
16    Kenya
17    Kenya
18    Kenya
19    Kenya
20    Kenya
21    Kenya
22    Kenya
23    Kenya
24    Kenya
25    Kenya
26    Kenya
27    Kenya
28    Kenya
29    Kenya
30    Kenya
31    Kenya
32    Kenya
33    Kenya
34    Kenya
35    Kenya
36    Kenya
37    Kenya
38    Kenya
39    Kenya
40    Kenya
41    Kenya
Name: country, dtype: object > File + Sheet: /content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx - SHEET= FOR IMPORT > Size: (42, 17)


Link:  Series([], Name: Dataset, dtype: object) 


Description: 

 Series([], Name: Dataset description, dtype: object) 




,COUNTY,COUNTY.1,DRR,Education,Emergency,Food,Governance,Health,logistics,Nutrition,Protection,Shelter,WaSH,Total,file,sheet,country
0,Baringo,BARINGO,1.0,2.0,1.0,1.0,NaN,1.0,NaN,NaN,2.0,1.0,2.0,5,kenya-3w-final-list-2017.xlsx,FOR IMPORT,Kenya
1,Bomet,BOMET,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,2,kenya-3w-final-list-2017.xlsx,FOR IMPORT,Kenya
2,Bungoma,BUNGOMA,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,2,kenya-3w-final-list-2017.xlsx,FOR IMPORT,Kenya
3,Busia,BUSIA,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,1.0,2,kenya-3w-final-list-2017.xlsx,FOR IMPORT,Kenya
4,Elgeyo-Marakwet,ELGEYO-MARAKWET,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,2.0,NaN,1.0,3,kenya-3w-final-list-2017.xlsx,FOR IMPORT,Kenya
5,Embu,EMBU,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,kenya-3w-final-list-2017.xlsx,FOR IMPORT,Kenya
6,Garissa,GARISSA,1.0,3.0,3.0,3.0,1.0,18.0,NaN,NaN,2.0,18.0,20.0,24,kenya-3w-final-list-2017.xlsx,FOR IMPORT,Kenya
7,Homa Bay,HOMA BAY,1.0,1.0,3.0,1.0,1.0,1.0,NaN,NaN,2.0,1.0,2.0,4,kenya-3w-final-list-2017.xlsx,FOR IMPORT,Kenya
8,Isiolo,ISIOLO,1.0,6.0,NaN,8.0,2.0,6.0,NaN,6.0,1.0,8.0,NaN,10,kenya-3w-final-list-2017.xlsx,FOR IMPORT,Kenya
9,Kajiado,KAJIADO,NaN,2.0,1.0,1.0,NaN,1.0,NaN,NaN,2.0,NaN,2.0,3,kenya-3w-final-list-2017.xlsx,FOR IMPORT,Kenya






/content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx - SHEET=  food import
Country: 0     Kenya
1     Kenya
2     Kenya
3     Kenya
4     Kenya
5     Kenya
6     Kenya
7     Kenya
8     Kenya
9     Kenya
10    Kenya
11    Kenya
12    Kenya
13    Kenya
14    Kenya
15    Kenya
16    Kenya
17    Kenya
18    Kenya
19    Kenya
20    Kenya
21    Kenya
22    Kenya
23    Kenya
24    Kenya
25    Kenya
26    Kenya
27    Kenya
28    Kenya
29    Kenya
30    Kenya
31    Kenya
32    Kenya
33    Kenya
34    Kenya
35    Kenya
36    Kenya
37    Kenya
38    Kenya
39    Kenya
40    Kenya
41    Kenya
Name: country, dtype: object > File + Sheet: /content/drive/MyDrive/Kenya/kenya-3w-final-list-2017.xlsx - SHEET=  food import > Size: (42, 6)


Link:  Series([], Name: Dataset, dtype: object) 


Description: 

 Series([], Name: Dataset description, dtype: object) 




...





/content/drive/MyDrive/Somalia/Somalia-Humanitarian Needs Overview-2020.xlsx - SHEET= Displaced
Country: 0     Somalia
1     Somalia
2     Somalia
3     Somalia
4     Somalia
       ...   
70    Somalia
71    Somalia
72    Somalia
73    Somalia
74    Somalia
Name: country, Length: 75, dtype: object > File + Sheet: /content/drive/MyDrive/Somalia/Somalia-Humanitarian Needs Overview-2020.xlsx - SHEET= Displaced > Size: (75, 31)


Link:  Series([], Name: Dataset, dtype: object) 


Description: 

 Series([], Name: Dataset description, dtype: object) 




,HUMANITARIAN CONSEQUENCE: Physical and mental well-being,Unnamed: 1,Unnamed: 2,Displaced,1011686.009,Unnamed: 5,Unnamed: 6,HUMANITARIAN CONSEQUENCE: Living standards,Unnamed: 8,Unnamed: 9,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Displaced.2,2545768.178,Unnamed: 27,file,sheet,country
0,District,Baseline Nondisplaced Population,"Acute malnutrition (Prevalence, not by severi...",Burden of mental healths issues,PiN - Wellbeing consequence,Selected indicator,NaN,District,Baseline Nondisplaced Population,multisectoral composite index,...,Households reporting inability to access finan...,Households reporting having being contacted by...,Households reporting awareness of procedures o...,Households reporting feeling that humanitarian...,Households reporting main concerns in accessin...,PiN - Recovery and resilience consequence,Selected indicator,Somalia-Humanitarian Needs Overview-2020.xlsx,Displaced,Somalia
1,Adan Yabaal,0.0,0.13904,0.327874,0,Burden of mental healths issues,NaN,Adan Yabaal,0.0,0.447368,...,0.900155,0.881966,0.851064,0.81308,0.422226,0,Households reporting inability to access finan...,Somalia-Humanitarian Needs Overview-2020.xlsx,Displaced,Somalia
2,Afgooye,56000.0,0.134586,0.259259,14518.51852,Burden of mental healths issues,NaN,Afgooye,56000.0,0.247423,...,0.938144,0.958763,0.938144,0.752577,0.453608,53690.72165,Households reporting awareness of procedures o...,Somalia-Humanitarian Needs Overview-2020.xlsx,Displaced,Somalia
3,Afmadow,68000.0,0.156969,0.820513,55794.87179,Burden of mental healths issues,NaN,Afmadow,68000.0,0.5,...,0.942308,0.826923,0.942308,0.903846,0.557692,64076.92308,Households reporting loss in income source,Somalia-Humanitarian Needs Overview-2020.xlsx,Displaced,Somalia
4,Baardheere,11000.0,0.157081,0.804878,8853.658537,Burden of mental healths issues,NaN,Baardheere,11000.0,0.631579,...,0.942308,0.826923,0.942308,0.930233,0.557692,10365.38462,Households reporting having being contacted by...,Somalia-Humanitarian Needs Overview-2020.xlsx,Displaced,Somalia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Wanla Weyne,10000.0,0.133963,0.820513,8205.128205,Burden of mental healths issues,NaN,Wanla Weyne,10000.0,0.704545,...,0.977273,1.0,1.0,0.931818,0.204545,10000,Households reporting earning income from preca...,Somalia-Humanitarian Needs Overview-2020.xlsx,Displaced,Somalia
71,Xarardheere,0.0,0.100963,0.096154,0,"Acute malnutrition (Prevalence, not by severi...",NaN,Xarardheere,0.0,0.444444,...,0.857143,0.730159,0.825397,0.777778,0.396825,0,Households reporting none or some children bei...,Somalia-Humanitarian Needs Overview-2020.xlsx,Displaced,Somalia
72,Xudun,18000.0,0.138,0.306452,5516.129032,Burden of mental healths issues,NaN,Xudun,18000.0,0.871429,...,0.957143,0.928571,0.971429,0.971429,0.2,17485.71429,Households reporting feeling that humanitarian...,Somalia-Humanitarian Needs Overview-2020.xlsx,Displaced,Somalia
73,Xudur,23000.0,0.125858,0.659091,15159.09091,Burden of mental healths issues,NaN,Xudur,23000.0,0.686275,...,0.980392,0.843137,0.862745,0.882353,0.686275,22549.01961,Households reporting having being contacted by...,Somalia-Humanitarian Needs Overview-2020.xlsx,Displaced,Somalia






/content/drive/MyDrive/Somalia/Somalia-Humanitarian Needs Overview-2020.xlsx - SHEET= Scoring table
Country: 0      Somalia
1      Somalia
2      Somalia
3      Somalia
4      Somalia
        ...   
155    Somalia
156    Somalia
157    Somalia
158    Somalia
159    Somalia
Name: country, Length: 160, dtype: object > File + Sheet: /content/drive/MyDrive/Somalia/Somalia-Humanitarian Needs Overview-2020.xlsx - SHEET= Scoring table > Size: (160, 10)


Link:  Series([], Name: Dataset, dtype: object) 


Description: 

 Series([], Name: Dataset description, dtype: object) 




,Outcome,Severity Class,None/ Minimal,Stress,Severe,Extreme,Catastrophic,file,sheet,country
0,NaN,Indicators,1.0,2.0,3.0,4.0,5.0,Somalia-Humanitarian Needs Overview-2020.xlsx,Scoring table,Somalia
1,Physical and mental well-being,NaN,NaN,NaN,NaN,NaN,NaN,Somalia-Humanitarian Needs Overview-2020.xlsx,Scoring table,Somalia
2,Prevalence of malnutrition,Acute malnutrition,NaN,NaN,NaN,NaN,NaN,Somalia-Humanitarian Needs Overview-2020.xlsx,Scoring table,Somalia
3,Mental health,Burden of mental healths issues,No issues reported & No history of chaining,1 - 3 issues reported OR history of chaining,At least 4 issues reported,At least 2 severe issues reported,At least 4 severe issues reported,Somalia-Humanitarian Needs Overview-2020.xlsx,Scoring table,Somalia
4,Living standards,NaN,NaN,NaN,NaN,NaN,NaN,Somalia-Humanitarian Needs Overview-2020.xlsx,Scoring table,Somalia
...,...,...,...,...,...,...,...,...,...,...
155,NaN,Households reporting main concerns in accessin...,No barrier,Not enough information,Cannot physically access points of humanitaria...,Insecurity on the way to humanitarian aid dist...,NaN,Somalia-Humanitarian Needs Overview-2020.xlsx,Scoring table,Somalia
156,NaN,NaN,NaN,NaN,NaN,OR,NaN,Somalia-Humanitarian Needs Overview-2020.xlsx,Scoring table,Somalia
157,NaN,NaN,NaN,NaN,NaN,Insecurity at points of humanitarian aid distr...,NaN,Somalia-Humanitarian Needs Overview-2020.xlsx,Scoring table,Somalia
158,NaN,NaN,NaN,NaN,NaN,OR,NaN,Somalia-Humanitarian Needs Overview-2020.xlsx,Scoring table,Somalia


None

## Loading pandas dataframes into a postgres database

Roughcode for how one might load all the dataframes into a SQL database. You can install postgres locally, or use docker and run [this](https://hub.docker.com/_/postgres).

In [ ]:
from sqlalchemy import create_engine  
import sqlalchemy

# Set these for your Postgres instance
host   = 'localhost' 
dbname = 'postgres'
user   = 'postgres'
pwd    = os.getenv['PWD'] # Don't save passwords in notebooks - that's naughty! :) - use environment variables. :)
port   = '5432'

engine = create_engine('postgresql+psycopg2://' + user + ':' + pwd + '@' + host +':' + port + '/' + dbname)

for k in dfs:
    df = dfs[k]
    table = k.replace('.','').replace(' ','_').replace('-','_').replace('/','').replace(',','')[0:62]
    print(f"Loading into database table {table} ...")
    df.to_sql(table, engine, if_exists='replace')

print("Done")

Loading into database table Kenyafts_requirements_funding_covid_kencsv ...
Loading into database table Kenyadbo_foodconsumptionscorescsv ...
Loading into database table Kenyaper_capita_food_consumption_2017_and_2018xlsx___Sheet1 ...
Loading into database table Kenyaretail_prices_for_dry_beans_2019_per_kgxlsx___Prices_for_ ...
Loading into database table Kenyavalue_of_recorded_marketed_agricultural_production_at_cur ...
Loading into database table Kenyawfp_food_prices_kencsv ...
Loading into database table Kenyafts_requirements_funding_cluster_kencsv ...
Loading into database table Kenyafts_incoming_funding_kencsv ...
Loading into database table Kenyahealth_kencsv ...
Loading into database table Kenyaken_adminboundaries_tabulardataxlsx___Admin2 ...
Loading into database table Kenyaken_adminboundaries_tabulardataxlsx___Admin1 ...
Loading into database table Kenyaken_adminboundaries_tabulardataxlsx___Admin0 ...
Loading into database table Kenyafts_requirements_funding_kencsv ...
Loading i

## Query from the database

In [ ]:
query = ' SELECT * FROM public."Kenyakenya_3w_final_list_2017xlsx___drr"'
df = pd.read_sql(query, con=engine)
display(df)

,index,Unnamed: 0,Unnamed: 1
0,0,Baringo,WVI
1,1,Embu,Trocaire
2,2,Garissa,ADESO
3,3,Homa Bay,PLAN
4,4,Isiolo,ADESO
5,5,Kilifi,PLAN
6,6,Kisumu,PLAN
7,7,Kitui,Trocaire
8,8,Kwale,PLAN
9,9,Machakos,PLAN


### Select all column names in the files which uploaded ok

In [ ]:
query = ' SELECT * FROM information_schema.columns'
df = pd.read_sql(query, con=engine)
display(df)

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,postgres,public,Kenyakenya_3w_final_list_2017xlsx___Table,Total,14,None,YES,double precision,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,postgres,pg_catalog,pg_statistic,stanumbers5,26,None,YES,ARRAY,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,postgres,pg_catalog,pg_statistic,stavalues1,27,None,YES,anyarray,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
3,postgres,pg_catalog,pg_statistic,stavalues2,28,None,YES,anyarray,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
4,postgres,pg_catalog,pg_statistic,stavalues3,29,None,YES,anyarray,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2843,postgres,information_schema,sql_parts,feature_id,1,None,YES,character varying,NaN,1.073742e+09,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2844,postgres,pg_catalog,pg_partitioned_table,partexprs,8,None,YES,pg_node_tree,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2845,postgres,information_schema,sql_parts,feature_name,2,None,YES,character varying,NaN,1.073742e+09,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2846,postgres,information_schema,sql_parts,is_supported,3,None,YES,character varying,3.0,1.200000e+01,...,NO,None,None,None,None,None,NO,NEVER,None,YES
